In [1]:
!pip install confluent-kafka

     |████████████████████████████████| 8.1 MB 12.0 MB/s eta 0:00:01


In [2]:
!pip install confluent-kafka[avro]

     |████████████████████████████████| 2.4 MB 21.8 MB/s eta 0:00:01
  Created wheel for avro-python3: filename=avro_python3-1.10.0-py3-none-any.whl size=43735 sha256=e34715d933828e23e7e7e1b0901afe551b3c9794bef82bb3e4a65e83c7f2563f
  Stored in directory: /home/jovyan/.cache/pip/wheels/52/db/5d/a6a6e9cd4c5d626432ce8258a0141de4f34cd7e68d79e9c4f4
Successfully built avro-python3


In [3]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [4]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [5]:
conf = {'bootstrap.servers': bootstrap_server}

In [6]:
a = AdminClient(conf)

In [7]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    ##if not (str(t)).startswith("_"):
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 8 topics:
  "_kafka-connect-01-offsets" with 25 partition(s)
  "_schemas" with 1 partition(s)
  "_kafka-connect-01-configs" with 1 partition(s)
  "quantia-analyticsksql_processing_log" with 1 partition(s)
  "__consumer_offsets" with 50 partition(s)
  "_kafka-connect-01-status" with 5 partition(s)
  "_confluent-ksql-quantia-analytics_command_topic" with 1 partition(s)
  "__transaction_state" with 50 partition(s)


In [8]:
#fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

#for topic, f in fs.items():
#    try:
#        f.result()  # The result itself is None
#        print("Topic {} created".format(topic))
#    except Exception as e:
#        print("Failed to create topic {}: {}".format(topic, e))

## Define the value schema

In [9]:
value_schema_str = """
{
  "namespace": "continuous.analytics",
  "type": "record",
  "name": "SmokeSensorEvent",
  "fields": [
    {
      "name": "sensor",
      "type": "string"
    },
    {
      "name": "smoke",
      "type": "boolean"
    },
    {
      "name": "ts",
      "type": "long"
    }
  ]
}
"""

In [10]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
import time

topic = "SmokeSensorEvent"
schema_registry_url = "http://schema-registry:8081"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [11]:
schema_registry_conf = {'url': schema_registry_url}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [12]:
avro_serializer = AvroSerializer(value_schema_str, schema_registry_client)

producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': avro_serializer
}

producer = SerializingProducer(producer_conf)

## run the following cell to demostrate that fire is not detected

In [ ]:

while True:
    key = "S1"
    value = {"sensor": "S1","smoke": False,"ts":int(time.time()*1000.0)}
    producer.produce(topic=topic, value=value, key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)


## run the following cell to demostrate to detect fire

In [ ]:

while True:
    key = "S1"
    value = {"sensor": "S1","smoke": True,"ts":int(time.time()*1000.0)}
    producer.produce(topic=topic, value=value, key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)

{'sensor': 'S1', 'smoke': True, 'ts': 1605624559352}
Produced record to topic SmokeSensorEvent partition [0] @ offset 66
{'sensor': 'S1', 'smoke': True, 'ts': 1605624569357}
Produced record to topic SmokeSensorEvent partition [0] @ offset 67
{'sensor': 'S1', 'smoke': True, 'ts': 1605624579364}
Produced record to topic SmokeSensorEvent partition [0] @ offset 68
{'sensor': 'S1', 'smoke': True, 'ts': 1605624589374}
Produced record to topic SmokeSensorEvent partition [0] @ offset 69
{'sensor': 'S1', 'smoke': True, 'ts': 1605624599385}
Produced record to topic SmokeSensorEvent partition [0] @ offset 70
{'sensor': 'S1', 'smoke': True, 'ts': 1605624609394}
Produced record to topic SmokeSensorEvent partition [0] @ offset 71
{'sensor': 'S1', 'smoke': True, 'ts': 1605624619394}
Produced record to topic SmokeSensorEvent partition [0] @ offset 72
{'sensor': 'S1', 'smoke': True, 'ts': 1605624629397}
Produced record to topic SmokeSensorEvent partition [0] @ offset 73
{'sensor': 'S1', 'smoke': True, 